In [4]:
import pandas as pd

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 199)
    
from datetime import datetime
import tensorflow
import keras
from keras import backend as K
import numpy as np
from keras.constraints import max_norm
from keras.optimizers import Adam
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold, KFold
from keras.layers import Dropout
from keras import regularizers

from keras.models import Sequential
from keras.layers import Dense, Input
from keras.layers.recurrent import LSTM
from keras.layers import Flatten
from sklearn.preprocessing import MinMaxScaler
import time
import tensorflow as tf
from sklearn.metrics import f1_score,roc_auc_score, log_loss, confusion_matrix

import seaborn as sns
from collections import Counter
import os

# %run "../../src/data_preparation/common_eda.py"

In [5]:
fold1 = pd.read_csv("../../data/processed/fold1Training_many_rows_raw.csv")
fold2 = pd.read_csv("../../data/processed/fold2Training_many_rows_raw.csv")
fold3 = pd.read_csv("../../data/processed/fold3Training_many_rows_raw.csv")
fold_submission = pd.read_csv("../../data/processed/testSet_many_rows_raw.csv")


fold1["type"] = "fold1"
fold2["type"] = "fold2"
fold3["type"] = "fold3"
fold_submission["type"] = "fold_sub"

In [6]:
feature_cols = ['TOTUSJH', 'TOTBSQ', 'TOTPOT', 'TOTUSJZ', 'ABSNJZH', 'SAVNCPP', 'USFLUX', 'TOTFZ', 'MEANPOT', 'EPSZ', 'MEANSHR', 'SHRGT45', 'MEANGAM', 'MEANGBT', 'MEANGBZ', 'MEANGBH', 'MEANJZH', 'TOTFY', 'MEANJZD', 'MEANALP', 'TOTFX', 'EPSY', 'EPSX', 'R_VALUE', 'XR_MAX']

In [7]:
def norm_data(df, feature_cols):
    scaler = MinMaxScaler(feature_range=(-1,1))

    for c in feature_cols:
        null_values = df[c].isnull()
        df.loc[~null_values, [c]] = scaler.fit_transform(df.loc[~null_values, [c]])
    return df

In [8]:
all_folds = pd.concat([fold1, fold2, fold3, fold_submission])
all_folds = norm_data(all_folds, feature_cols)
all_folds.shape

(22186320, 28)

In [9]:
list_df = [fold1, fold2, fold3, fold_submission]


In [10]:
for df in list_df:
    print(len(df))
    print(df["ID"].nunique())
    print("------------")

4606380
76773
------------
5548860
92481
------------
1620360
27006
------------
10410720
173512
------------


In [11]:
fold1 = all_folds[all_folds.type == "fold1"]
fold2 = all_folds[all_folds.type == "fold2"]
fold3 = all_folds[all_folds.type == "fold3"]
fold_submission = all_folds[all_folds.type == "fold_sub"]

In [12]:
for df in list_df:
    print(len(df))
    print(df["ID"].nunique())
    print("------------")

4606380
76773
------------
5548860
92481
------------
1620360
27006
------------
10410720
173512
------------


In [13]:
fold1.drop("type", 1).to_pickle("../../data/processed/fold1_norm_new.pkl")
fold2.drop("type", 1).to_pickle("../../data/processed/fold2_norm_new.pkl")
fold3.drop("type", 1).to_pickle("../../data/processed/fold3_norm_new.pkl")
fold_submission.drop("type", 1).to_pickle("../../data/processed/fold_submission_norm_new.pkl")

In [14]:
fold1.head()

,TOTUSJH,TOTBSQ,TOTPOT,TOTUSJZ,ABSNJZH,SAVNCPP,USFLUX,TOTFZ,MEANPOT,EPSZ,MEANSHR,SHRGT45,MEANGAM,MEANGBT,MEANGBZ,MEANGBH,MEANJZH,TOTFY,MEANJZD,MEANALP,TOTFX,EPSY,EPSX,R_VALUE,XR_MAX,LABEL,ID,type
0,-0.995516,-0.995142,-0.999954,-0.996614,-0.993539,-0.991184,-0.989615,0.747205,-0.999993,-0.632636,-0.671403,-0.923798,-0.463852,-0.598104,-0.501369,-0.776450,-0.156038,0.284461,-0.036956,0.411820,0.112060,0.078960,-0.554679,-0.436045,1.0,0.0,1,fold1
1,-0.995657,-0.995010,-0.999951,-0.996723,-0.992226,-0.986662,-0.989511,0.747057,-0.999993,-0.623829,-0.666793,-0.916873,-0.445325,-0.596575,-0.497341,-0.773209,-0.152085,0.283627,-0.033033,0.411949,0.112060,0.103436,-0.539360,-1.000000,1.0,0.0,1,fold1
2,-0.995390,-0.994890,-0.999949,-0.996458,-0.990422,-0.981634,-0.989220,0.746763,-0.999993,-0.624441,-0.661389,-0.878897,-0.446607,-0.595262,-0.496909,-0.765827,-0.147467,0.282799,-0.039246,0.412123,0.112186,0.126763,-0.533032,-0.231710,1.0,0.0,1,fold1
3,-0.995321,-0.994943,-0.999952,-0.996467,-0.993251,-0.988751,-0.989254,0.746785,-0.999993,-0.629688,-0.667716,-0.892921,-0.438189,-0.596369,-0.499920,-0.766752,-0.156144,0.282744,-0.030963,0.411823,0.111907,0.129788,-0.523273,-0.225941,1.0,0.0,1,fold1
4,-0.995592,-0.994964,-0.999953,-0.996649,-0.993603,-0.989495,-0.989117,0.746662,-0.999993,-0.638807,-0.671921,-0.899941,-0.454324,-0.604588,-0.514620,-0.774688,-0.157397,0.282917,-0.031986,0.411784,0.111952,0.124867,-0.528127,-1.000000,1.0,0.0,1,fold1


In [15]:
fold2 = pd.read_pickle("../../data/processed/fold2_norm_new.pkl")
fold2.head()

,TOTUSJH,TOTBSQ,TOTPOT,TOTUSJZ,ABSNJZH,SAVNCPP,USFLUX,TOTFZ,MEANPOT,EPSZ,MEANSHR,SHRGT45,MEANGAM,MEANGBT,MEANGBZ,MEANGBH,MEANJZH,TOTFY,MEANJZD,MEANALP,TOTFX,EPSY,EPSX,R_VALUE,XR_MAX,LABEL,ID
0,-0.971583,-0.962050,-0.999434,-0.979271,-0.993873,-0.969246,-0.950882,0.712394,-0.999984,-0.326254,-0.460832,-0.423182,0.052041,-0.602045,-0.522307,-0.627449,-0.179961,0.230664,-0.028236,0.410982,0.101181,0.236347,0.031804,0.099388,1.0,0.0,1
1,-0.972222,-0.961889,-0.999436,-0.979310,-0.992911,-0.984325,-0.950330,0.711719,-0.999985,-0.329554,-0.462582,-0.432794,0.054625,-0.605208,-0.526397,-0.633112,-0.180422,0.230641,-0.031623,0.410965,0.101414,0.235449,0.030058,0.082380,1.0,0.0,1
2,-0.971711,-0.961116,-0.999413,-0.978913,-0.996036,-0.977690,-0.949592,0.711901,-0.999984,-0.321856,-0.455067,-0.402011,0.055054,-0.612272,-0.532340,-0.637010,-0.178559,0.230224,-0.030315,0.411022,0.100954,0.232494,0.033290,0.131759,1.0,0.0,1
3,-0.973200,-0.961085,-0.999407,-0.979454,-0.995930,-0.985531,-0.949965,0.712682,-0.999984,-0.316321,-0.452943,-0.410777,0.065433,-0.614244,-0.535105,-0.637266,-0.178629,0.230772,-0.031934,0.411020,0.100122,0.230073,0.039305,0.115243,1.0,0.0,1
4,-0.972440,-0.961011,-0.999413,-0.978900,-0.998068,-0.991689,-0.949708,0.712295,-0.999984,-0.318347,-0.452915,-0.400112,0.070654,-0.616053,-0.538056,-0.640108,-0.177392,0.230726,-0.033250,0.411057,0.099359,0.229830,0.044796,0.064870,1.0,0.0,1


In [16]:
fold_submission.head()

,TOTUSJH,TOTBSQ,TOTPOT,TOTUSJZ,ABSNJZH,SAVNCPP,USFLUX,TOTFZ,MEANPOT,EPSZ,MEANSHR,SHRGT45,MEANGAM,MEANGBT,MEANGBZ,MEANGBH,MEANJZH,TOTFY,MEANJZD,MEANALP,TOTFX,EPSY,EPSX,R_VALUE,XR_MAX,LABEL,ID,type
0,-0.993511,-0.989019,-0.999929,-0.995608,-0.996709,-0.986189,-0.977875,0.729270,-0.999995,-0.712280,-0.721893,-0.992931,-0.589750,-0.710999,-0.650902,-0.822202,-0.170998,0.293508,-0.042045,0.411247,0.117593,-0.095089,-0.373166,-0.003025,1.0,NaN,1,fold_sub
1,-0.993760,-0.989204,-0.999930,-0.995827,-0.996740,-0.983936,-0.978310,0.729825,-0.999994,-0.710856,-0.722814,-0.986129,-0.593397,-0.708514,-0.647445,-0.818987,-0.170902,0.293512,-0.037520,0.411249,0.116945,-0.096797,-0.363091,0.013228,1.0,NaN,1,fold_sub
2,-0.994078,-0.989442,-0.999930,-0.995992,-0.996746,-0.986993,-0.978563,0.730708,-0.999995,-0.704813,-0.721547,-0.989522,-0.589012,-0.712097,-0.652921,-0.820998,-0.170956,0.294063,-0.040533,0.411253,0.116822,-0.107296,-0.368565,0.054617,1.0,NaN,1,fold_sub
3,-0.994179,-0.989560,-0.999932,-0.995978,-0.994405,-0.981601,-0.978906,0.730844,-0.999995,-0.709264,-0.719631,-0.991973,-0.587419,-0.708522,-0.649580,-0.820592,-0.166912,0.293795,-0.039541,0.411371,0.116988,-0.104448,-0.377496,0.001792,1.0,NaN,1,fold_sub
4,-0.993849,-0.989599,-0.999930,-0.995759,-0.996004,-0.984986,-0.978806,0.731167,-0.999995,-0.703704,-0.714837,-0.977785,-0.577609,-0.711626,-0.653305,-0.819350,-0.169751,0.293921,-0.039131,0.411292,0.116991,-0.106765,-0.379060,0.023437,1.0,NaN,1,fold_sub


In [ ]:
c=4
sns.distplot(fold1.iloc[:,c].fillna(0))
sns.distplot(fold3.iloc[:,c].fillna(0))

In [ ]:
c=5
sns.distplot(fold1.iloc[:,c].fillna(0))
sns.distplot(fold3.iloc[:,c].fillna(0))

In [ ]:
c=6
sns.distplot(fold1.iloc[:,c].fillna(0))
sns.distplot(fold3.iloc[:,c].fillna(0))

In [ ]:
c=7
sns.distplot(fold1.iloc[:,c].fillna(0))
sns.distplot(fold3.iloc[:,c].fillna(0))

In [ ]:
c=8
sns.distplot(fold1.iloc[:,c].fillna(0))
sns.distplot(fold3.iloc[:,c].fillna(0))

In [ ]:
c=9
sns.distplot(fold1.iloc[:,c].fillna(0))
sns.distplot(fold3.iloc[:,c].fillna(0))

In [ ]:
c=10
sns.distplot(fold1.iloc[:,c].fillna(0))
sns.distplot(fold3.iloc[:,c].fillna(0))

In [ ]:
c=11
sns.distplot(fold1.iloc[:,c].fillna(0))
sns.distplot(fold3.iloc[:,c].fillna(0))

In [ ]:
c=12
sns.distplot(fold1.iloc[:,c].fillna(0))
sns.distplot(fold3.iloc[:,c].fillna(0))

In [ ]:
c=13
sns.distplot(fold1.iloc[:,c].fillna(0))
sns.distplot(fold3.iloc[:,c].fillna(0))

In [ ]:
c=14
sns.distplot(fold1.iloc[:,c].fillna(0))
sns.distplot(fold3.iloc[:,c].fillna(0))

In [ ]:
c=15
sns.distplot(fold1.iloc[:,c].fillna(0))
sns.distplot(fold3.iloc[:,c].fillna(0))

In [ ]:
c=16
sns.distplot(fold1.iloc[:,c].fillna(0))
sns.distplot(fold3.iloc[:,c].fillna(0))

In [ ]:
c=17
sns.distplot(fold1.iloc[:,c].fillna(0))
sns.distplot(fold3.iloc[:,c].fillna(0))

In [ ]:
c=18
sns.distplot(fold1.iloc[:,c].fillna(0))
sns.distplot(fold3.iloc[:,c].fillna(0))

In [ ]:
c=19

sns.distplot(fold1.iloc[:,c].fillna(0))
sns.distplot(fold3.iloc[:,c].fillna(0))

In [ ]:
c=20
sns.distplot(fold1.iloc[:,c].fillna(0))
sns.distplot(fold3.iloc[:,c].fillna(0))

In [ ]:
c=21
sns.distplot(fold1.iloc[:,c].fillna(0))
sns.distplot(fold3.iloc[:,c].fillna(0))

In [ ]:
c=22
sns.distplot(fold1.iloc[:,c].fillna(0))
sns.distplot(fold3.iloc[:,c].fillna(0))

In [ ]:
c=23
sns.distplot(fold1.iloc[:,c].fillna(0))
sns.distplot(fold3.iloc[:,c].fillna(0))

In [ ]:
c=24
sns.distplot(fold1.iloc[:,c].fillna(0))
sns.distplot(fold3.iloc[:,c].fillna(0))

In [ ]:
c=25
sns.distplot(fold1.iloc[:,c].fillna(0))
sns.distplot(fold3.iloc[:,c].fillna(0))